In [13]:
import pickle
import numpy as np
from scipy import sparse
import pandas as pd
import seaborn as sns
sns.set() ## set up style
import uuid
import matplotlib.pyplot as plt
#solved chinese display in matplotlib
from matplotlib import rcParams
rcParams['font.family'] = 'sans-serif'
rcParams["font.family"] = "DFKai-SB"
rcParams['axes.unicode_minus'] = False

import torch
import visdom
vis=visdom.Visdom()
from BuildDataset import tripletdataset
from Model import Encoder,Decoder
industry=['金融業', 'IC製造', 'IC設計', '電子零件元件', '電腦系統業', '石油及天然氣', '手機', '面板業', '太陽能',
       '板鋼', '被動元件', '車輛整車', '非鐵金屬', '流通業', '地產', '通訊設備', '遊戲產業', '印刷電路板相關',
       '貴金屬', '機械', '運輸事業', '電子通路', 'IC封裝測試', 'LED', '通訊服務', '消費性電子產品', '化學工業',
       '醫藥產業', '軟體業', '週邊產品', '石化業', '旅館、餐飲', '汽機車零組件', '設備儀器廠商', '條鋼', '造紙業',
       '水泥', '橡膠工業', '傳產其他', '數位相機', '顯示器', '不鏽鋼', '農林漁牧', '服務業', 'Internet相關',
       '營造工程', '休閒娛樂', '紡織中游', '生物科技', '家電', '光碟片', '建材', '成衣', '電力', '運動產業',
       '線材、盤元', '化纖原料', '其他公用事業', '大宗物資', '家居用品', '手機零組件', '食品加工', '電力設備',
       '航天軍工', '分離式元件', '電子其他', '面板零組件', '飲料相關', '電線電纜', '封測服務與材料', '礦石開採',
       '光通訊', '傳播事業', '輔助與彌補用醫材', '時尚產業', '基礎建設營運', '電聲產品', '水資源', '醫療器材通路',
       '合金鋼', '文化創意產業', '車用金屬成型', '穿戴式裝置', '其他醫療器材', '汽車內裝', '車用電子',
       '診斷與監測用醫材', '傳輸介面', '電子化工材料', '電池材料相關', '資產股', '醫療管理服務', '體外診斷用醫材', '煤',
       '射頻前端晶片', '控股公司', '無店舖販售', '手術與治療用醫材', '金屬礦採選', '生物辨識相關']

In [3]:
## Load industry dictionary
with open("D:4.AutoencoderForArticle.Mid-Industry/dict_guid_sparsebow_single.v05.p","rb") as f:
    dict_guid_sparsebow_single= pickle.load(f)
with open("D:4.AutoencoderForArticle.Mid-Industry/dict_singleindustry_guid.p","rb") as f:
    dict_singleindustry_guid= pickle.load(f)

In [22]:
## Industry with news more than 100 pics
industry_partial=['金融業', 'IC製造', 'IC設計', '電子零件元件', '電腦系統業', '石油及天然氣', '手機', '面板業', '太陽能',
       '板鋼', '被動元件', '車輛整車', '非鐵金屬', '流通業', '地產', '通訊設備', '遊戲產業', '印刷電路板相關',
       '貴金屬', '機械', '運輸事業', '電子通路', 'IC封裝測試', 'LED', '通訊服務', '消費性電子產品', '化學工業',
       '醫藥產業', '軟體業', '週邊產品', '石化業', '旅館、餐飲', '汽機車零組件', '設備儀器廠商', '條鋼', '造紙業',
       '水泥', '橡膠工業', '傳產其他', '數位相機', '顯示器', '不鏽鋼', '農林漁牧', '服務業', 'Internet相關',
       '營造工程', '休閒娛樂', '紡織中游', '生物科技', '家電', '光碟片', '建材', '成衣', '電力', '運動產業',
       '線材、盤元', '化纖原料', '其他公用事業', '大宗物資', '家居用品', '手機零組件', '食品加工', '電力設備',
       '航天軍工', '分離式元件', '電子其他', '面板零組件', '飲料相關', '電線電纜', '封測服務與材料', '礦石開採',
       '光通訊', '傳播事業', '輔助與彌補用醫材', '時尚產業', '基礎建設營運', '電聲產品', '水資源', '醫療器材通路',
       '合金鋼']

#### Configuration

In [56]:
Split=5/18
Split_train_test=3/10

In [58]:
dict_split_industry_guid={ ind:np.random.choice(dict_singleindustry_guid[ind],
                                                     size=int(Split*len(dict_singleindustry_guid[ind])),
                                                     replace=False) for ind in industry_partial}

In [59]:
analysis={i:len(dict_5of18_industry_guid[i]) for i in dict_5of18_industry_guid }
analysis=pd.Series(analysis)
analysis.sort_values(ascending=False)

金融業         5902
IC製造        2781
IC設計        2655
電子零件元件      2443
電腦系統業       1901
石油及天然氣      1800
手機          1782
面板業         1607
太陽能         1516
板鋼          1380
被動元件        1369
車輛整車        1359
非鐵金屬        1343
流通業         1283
地產          1160
通訊設備        1140
遊戲產業        1117
印刷電路板相關     1083
貴金屬          978
機械           898
運輸事業         897
電子通路         840
IC封裝測試       754
LED          665
通訊服務         664
消費性電子產品      589
化學工業         581
醫藥產業         559
軟體業          557
週邊產品         557
            ... 
光碟片          190
建材           164
成衣           163
電力           155
運動產業         151
線材、盤元        150
化纖原料         146
其他公用事業       140
大宗物資         137
家居用品         131
手機零組件        115
食品加工         107
電力設備         102
航天軍工          97
分離式元件         95
電子其他          90
面板零組件         85
飲料相關          78
電線電纜          76
封測服務與材料       75
礦石開採          65
光通訊           52
傳播事業          50
時尚產業          50
輔助與彌補用醫材      50
基礎建設營運        45
電聲產品          43
水資源           

In [60]:
analysis={i:pd.Series(dict_split_industry_guid[i]).unique() for i in dict_split_industry_guid }
analysis

{'IC封裝測試': array([UUID('059d56d6-c315-496f-8380-29a266bdea43'),
        UUID('c3362b8b-4a68-4851-9c9d-65e299ef8304'),
        UUID('12fb2c7a-03f6-4ea7-a823-f2a3f18d3fdb'),
        UUID('4ee5d465-4008-427d-aa73-df47bea30981'),
        UUID('cf5616c9-ba29-4aa8-8fe7-c51c954edef4'),
        UUID('2e6a9f1a-a14e-4819-b8f5-e4e5cc6273e4'),
        UUID('02ae7c5f-e27d-4453-992d-3730077875dd'),
        UUID('8b4999ad-51a3-4463-b680-b33dcfff7e13'),
        UUID('db3d787c-84c2-431a-bfc7-8da6b94d7b0a'),
        UUID('32a99bc0-f086-4d15-96ef-910b02a3038d'),
        UUID('fff8d0b6-adef-4a68-827f-a5c18e90ae73'),
        UUID('f85740b0-6302-4cf9-ad63-85c604011ea1'),
        UUID('d9b41e0d-5d22-4f86-9674-5dca6b8f0fcf'),
        UUID('fdebdd44-ae55-4086-8f40-8c1226ed6f1c'),
        UUID('1ed8472f-d21e-409b-8b85-5121001f8cc3'),
        UUID('348c59cc-ef92-4e38-bd02-f166cb26d8f8'),
        UUID('b8f0ef85-7a09-450c-a14e-71e91e6f0c67'),
        UUID('d7f7c3d7-1950-44ba-93ed-749a2b21b9fa'),
        UUID('2b9e

In [ ]:
Epochs=100
for i,_  

In [7]:
model=Encoder(70000,500)

In [8]:
torch.save(model, "test")

In [16]:
torch.load("model/test")

Encoder(
  (linear1): Linear(in_features=70000, out_features=2000, bias=True)
  (linear2): Linear(in_features=2000, out_features=500, bias=True)
  (sigmoid): Sigmoid()
  (tanh): Tanh()
)

In [ ]:
for i in 

In [ ]:
### PLOT

In [145]:
win="Total-loss"
opts={"markers":True,"xlabel":"epochs","title":"Total Loss","width":1200,"heigh":300,"xtickmax":100}
y=torch.randn(3)
win=vis.line(y,X=[1,2,3],win=win,opts=opts,name="train-loss")

In [141]:
y=torch.randn(3)
win=vis.line(y,X=[3,5,8],win=win,update="append",opts=opts,name="test-loss")

In [142]:
win="test"
opts={"markers":True,"xlabel":"epochs","title":"test Loss","width":1200,"heigh":300}
y=torch.randn(3)
win=vis.line(y,X=[1,2,3],win=win,opts=opts,name="train-loss")

In [143]:
win="X"
opts={"markers":True,"xlabel":"epochs","title":"X Loss","width":1200,"heigh":300}
y=torch.randn(3)
win=vis.line(y,X=[1,2,3],win=win,opts=opts,name="train-loss")